### In this notebook, we annotated topics data using majority vote, with preferring gpt4o labels

In [1]:
import os
import pandas as pd
from typing import List, Dict
from rich import print
from collections import Counter

In [2]:
train_gpt4o = pd.read_csv('../../data/result/train_with_topic_df_llm_gpt4o.csv')
test_gpt4o = pd.read_csv('../../data/result/test_with_topic_df_llm_gpt4o.csv')

train_command = pd.read_csv('../../data/result/train_with_topic_df_llm_command_r_plus.csv')
test_command = pd.read_csv('../../data/result/test_with_topic_df_llm_command_r_plus.csv')

train_llama3 = pd.read_csv('../../data/result/train_with_topic_df_llm_llama_3.csv')
test_llama3 = pd.read_csv('../../data/result/test_with_topic_df_llm_llama_3.csv')

In [3]:
def majority_vote(labels_dict: Dict, labels_list: List[str], break_tie: List):
    """
    Determines the majority vote from multiple annotators' labels for a given example.

    Parameters:
    -----------
    labels_dict : dict
        A dictionary with annotator names as keys and their corresponding labels as values.
    labels_list : list
        A list containing all possible labels.
    break_tie : str
        The annotator whose label should be favored in case of a tie. If `break_tie` is not a key in `labels_dict`, its value will be preferred if it is in `labels_list`.

    Returns:
    --------
    str
        The label that represents the majority vote.
    bool
        If there is a tie or not.
    """
    labels = list(labels_dict.values())
    
    tie = False
    # break tie
    if len(set(labels)) == len(labels):
        label = labels_dict.get(break_tie, break_tie)
        tie = True
        if label not in labels_list:
            raise ValueError(f"The label '{label}' is not in the list of possible labels.")
    
    else:
        label = Counter(labels).most_common()[0][0]

    return label, tie

In [4]:
train_command.head()

,column,text,topic_id,topic_name,topic_llm
0,like,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه...,3,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري
1,improve_course,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,3,موازنة الجزء العملي مع الجزء النظري,طريقة تدريس و تقديم المادة
2,like,أكثر ما أعجبني: the content,0,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر
3,improve_course,اقتراحاتي للتحسين: provide more new books,3,موازنة الجزء العملي مع الجزء النظري,غير محدد
4,improve_course,اقتراحاتي للتحسين: زيادة ساعات العملي,8,الوقت و الجدول,موازنة الجزء العملي مع الجزء النظري


In [5]:
labels_list = list(train_gpt4o['topic_name'].apply(lambda x: "غير محدد" if x == 'None' else x).unique())

In [6]:
print(labels_list)

[
    'موازنة الجزء العملي مع الجزء النظري',
    'محتوى ومعلومات المقرر',
    'الوقت و الجدول',
    'دكتور المقرر',
    'طريقة تدريس و تقديم المادة',
    'غير محدد',
    'سوق العمل',
    'المتطلبات و المهام و الدرجات',
    'الجانب البحثي و الأوراق البحثية'
]

In [7]:
def get_label(label_cols: List[List[str]], break_tie: str, labels_list: List[str]):
    labels = []
    ties = []
    for i, (gpt_label, llama_label, command_label) in enumerate(zip(*label_cols)):
        labels_dict = {}
        labels_dict['gpt'] = gpt_label
        labels_dict['llama3'] = llama_label
        labels_dict['command'] = command_label

        label, tie = majority_vote(labels_dict, labels_list, break_tie=break_tie)

        labels.append(label)
        ties.append(tie)
    
    return labels, ties

In [8]:
train_gpt_labels = train_gpt4o['topic_llm']
train_llama_labels = train_llama3['topic_llm']
train_command_labels = train_command['topic_llm']

test_gpt_labels = test_gpt4o['topic_llm']
test_llama_labels = test_llama3['topic_llm']
test_command_labels = test_command['topic_llm']

In [9]:
train_labels, train_ties = get_label([train_gpt_labels, train_llama_labels, train_command_labels], break_tie='gpt', labels_list=labels_list)

In [10]:
test_labels, test_ties = get_label([test_gpt_labels, test_llama_labels, test_command_labels], break_tie='gpt', labels_list=labels_list)

In [11]:
train_gpt4o.head()

,column,text,topic_id,topic_name,topic_llm
0,like,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه...,3,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري
1,improve_course,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,3,موازنة الجزء العملي مع الجزء النظري,طريقة تدريس و تقديم المادة
2,like,أكثر ما أعجبني: the content,0,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر
3,improve_course,اقتراحاتي للتحسين: provide more new books,3,موازنة الجزء العملي مع الجزء النظري,محتوى ومعلومات المقرر
4,improve_course,اقتراحاتي للتحسين: زيادة ساعات العملي,8,الوقت و الجدول,موازنة الجزء العملي مع الجزء النظري


In [12]:
train_gpt4o['majority_vote'] = train_labels
train_gpt4o['tie'] = train_ties

In [13]:
test_gpt4o['majority_vote'] = test_labels
test_gpt4o['tie'] = test_ties

In [14]:
train_gpt4o.head()

,column,text,topic_id,topic_name,topic_llm,majority_vote,tie
0,like,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه...,3,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري,False
1,improve_course,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,3,موازنة الجزء العملي مع الجزء النظري,طريقة تدريس و تقديم المادة,طريقة تدريس و تقديم المادة,False
2,like,أكثر ما أعجبني: the content,0,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر,False
3,improve_course,اقتراحاتي للتحسين: provide more new books,3,موازنة الجزء العملي مع الجزء النظري,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر,False
4,improve_course,اقتراحاتي للتحسين: زيادة ساعات العملي,8,الوقت و الجدول,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري,False


In [15]:
test_gpt4o.head()

,column,text,topic_id,topic_name,topic_llm,majority_vote,tie
0,like,أكثر ما أعجبني: المقرر ساعدني جدا في فهم نظام ...,1,دكتور المقرر,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري,False
1,dislike,أكثر ما لم يعجبني: المقرر يحتوي على مواضيع ساب...,6,طريقة تدريس و تقديم المادة,المتطلبات و المهام و الدرجات,المتطلبات و المهام و الدرجات,False
2,dislike,أكثر ما لم يعجبني: أسلوب شرح المقرر,5,المتطلبات و المهام و الدرجات,طريقة تدريس و تقديم المادة,طريقة تدريس و تقديم المادة,False
3,add_suggestions,اقتراحاتي للإضافة: everything is spectacular a...,2,None,غير محدد,غير محدد,False
4,like,أكثر ما أعجبني: معلمة المادة متمكنة وتستجيب عل...,1,دكتور المقرر,دكتور المقرر,دكتور المقرر,False


In [16]:
# ties len
sum(train_ties), sum(test_ties)

(36, 32)

In [17]:
train_save_path = '../../data/result/train_with_topic_df_majority_vote_gpt4o_preferred.csv'
if os.path.exists(train_save_path):
    print('The path exists!')
else:
    print(f'Saving to {train_save_path}...')
    train_gpt4o.to_csv(train_save_path, index=False)


test_save_path = '../../data/result/test_with_topic_df_majority_vote_gpt4o_preferred.csv'
if os.path.exists(test_save_path):
    print('The path exists!')
else:
    print(f'Saving to {test_save_path}...')
    test_gpt4o.to_csv(test_save_path, index=False)

The path exists!

The path exists!

In [18]:
import pandas as pd

In [19]:
pd.read_csv('../../data/result/train_with_topic_df_majority_vote_gpt4o_preferred.csv')['majority_vote'].value_counts()


محتوى ومعلومات المقرر                  226
دكتور المقرر                           176
موازنة الجزء العملي مع الجزء النظري    166
غير محدد                               131
المتطلبات و المهام و الدرجات            74
طريقة تدريس و تقديم المادة              73
الوقت و الجدول                          39
الجانب البحثي و الأوراق البحثية         35
سوق العمل                               31
Name: majority_vote, dtype: int64

# Convert to HF dataset

In [20]:
from datasets import Dataset, Value, ClassLabel, Features, DatasetDict, load_dataset, load_from_disk
import pandas as pd

In [21]:
train_path = '../../data/result/train_with_topic_df_majority_vote_gpt4o_preferred.csv'

In [22]:
data = pd.read_csv(train_path)

In [23]:
data.head()

,column,text,topic_id,topic_name,topic_llm,majority_vote,tie
0,like,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه...,3,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري,False
1,improve_course,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,3,موازنة الجزء العملي مع الجزء النظري,طريقة تدريس و تقديم المادة,طريقة تدريس و تقديم المادة,False
2,like,أكثر ما أعجبني: the content,0,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر,False
3,improve_course,اقتراحاتي للتحسين: provide more new books,3,موازنة الجزء العملي مع الجزء النظري,محتوى ومعلومات المقرر,محتوى ومعلومات المقرر,False
4,improve_course,اقتراحاتي للتحسين: زيادة ساعات العملي,8,الوقت و الجدول,موازنة الجزء العملي مع الجزء النظري,موازنة الجزء العملي مع الجزء النظري,False


In [24]:
data = data.drop(columns=['column', 'topic_id', 'topic_name', 'topic_llm', 'tie'])
data = data.rename(columns={'majority_vote': 'labels'})

In [25]:
data.head()

,text,labels
0,أكثر ما أعجبني: التطبيق العملي المحدث والمشابه...,موازنة الجزء العملي مع الجزء النظري
1,اقتراحاتي للتحسين: التطبيق والبعد عن التدريس ا...,طريقة تدريس و تقديم المادة
2,أكثر ما أعجبني: the content,محتوى ومعلومات المقرر
3,اقتراحاتي للتحسين: provide more new books,محتوى ومعلومات المقرر
4,اقتراحاتي للتحسين: زيادة ساعات العملي,موازنة الجزء العملي مع الجزء النظري


In [26]:
topic_names = sorted(data['labels'].unique().tolist())
topic_names

['الجانب البحثي و الأوراق البحثية',
 'المتطلبات و المهام و الدرجات',
 'الوقت و الجدول',
 'دكتور المقرر',
 'سوق العمل',
 'طريقة تدريس و تقديم المادة',
 'غير محدد',
 'محتوى ومعلومات المقرر',
 'موازنة الجزء العملي مع الجزء النظري']

In [27]:
features = Features(
    {
        'id': Value(dtype='string'),
        'text': Value(dtype='string'),
        'labels': ClassLabel(names=topic_names, num_classes=len(topic_names), id=None)
    }
)

print(features)

{
    'id': Value(dtype='string', id=None),
    'text': Value(dtype='string', id=None),
    'labels': ClassLabel(
        names=[
            'الجانب البحثي و الأوراق البحثية',
            'المتطلبات و المهام و الدرجات',
            'الوقت و الجدول',
            'دكتور المقرر',
            'سوق العمل',
            'طريقة تدريس و تقديم المادة',
            'غير محدد',
            'محتوى ومعلومات المقرر',
            'موازنة الجزء العملي مع الجزء النظري'
        ],
        id=None
    )
}

In [28]:
hf_dataset = DatasetDict({
    'train': Dataset.from_dict(
        {
            'id': list(data.index.astype(str)),
            'text': list(data['text']),
            'labels': list(data['labels'])
        },
        features=features
    )
})

In [29]:
hf_path = '../../data/result/topics_hf_dataset'

In [30]:
hf_dataset.save_to_disk(hf_path)

Saving the dataset (0/1 shards):   0%|          | 0/951 [00:00<?, ? examples/s]